In [1]:
import cPickle
import numpy as np
import pandas as pd
import datetime
from sklearn import preprocessing
from sklearn.model_selection import TimeSeriesSplit

import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn import neighbors
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
import operator
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, classification_report

from pandas_datareader import data, wb
import pandas_datareader as pdr
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
import re
from dateutil import parser
from bt import Strategy#, Portfolio
import quandl

In [2]:
def getStock(symbol, start, end):
    """
    Downloads Stock from Yahoo Finance.
    Computes daily Returns based on Adj Close.
    Returns pandas dataframe.
    """
    
    df =  pdr.data.get_data_yahoo(symbol, start, end)

    df.columns.values[-1] = 'AdjClose'
    df.columns = df.columns + '_' + symbol
    df['Return_%s' %symbol] = df['AdjClose_%s' %symbol].pct_change()
    
    print(df)
    
    return df

d1 = datetime.datetime(2000,5,1)
d2 = datetime.datetime(2017,5,1)

df = getStock("GOOG", d1, d2)

df.columns

             Open_GOOG   High_GOOG    Low_GOOG  Close_GOOG  Volume_GOOG  \
Date                                                                      
2004-08-19  100.000168  104.060182   95.960165  100.340176     44871300   
2004-08-20  101.010175  109.080187  100.500174  108.310183     22942800   
2004-08-23  110.750191  113.480193  109.050183  109.400185     18342800   
2004-08-24  111.240189  111.600192  103.570177  104.870176     15319700   
2004-08-25  104.960181  108.000187  103.880180  106.000184      9232100   
2004-08-26  104.950180  107.950188  104.660179  107.910182      7128600   
2004-08-27  108.100185  108.620186  105.690180  106.150181      6241200   
2004-08-30  105.280178  105.490184  102.010172  102.010172      5221400   
2004-08-31  102.300173  103.710180  102.160177  102.370175      4941200   
2004-09-01  102.700174  102.970180   99.670169  100.250171      9181600   
2004-09-02   99.190173  102.370175   98.940170  101.510173     15190400   
2004-09-03  100.950174  1

Index([u'Open_GOOG', u'High_GOOG', u'Low_GOOG', u'Close_GOOG', u'Volume_GOOG',
       u'AdjClose_GOOG', u'Return_GOOG'],
      dtype='object')

In [3]:
def getStockDataFromWeb(fout, start_string, end_string):
    """
    Collects predictors data from Yahoo Finance and Quandl.
    Returns a list of dataframes.
    """
    start = start_string
    end = end_string
    
    sp500 = getStock('^GSPC', start, end)
    nasdaq = getStock('^IXIC', start, end)
    frankfurt = getStock('^GDAXI', start, end)
    london = getStock('^FTSE', start, end)
    paris = getStock('^FCHI', start, end)
    hkong = getStock('^HSI', start, end)
    nikkei = getStock('^N225', start, end)
    australia = getStock('^AXJO', start, end)
    djia = getStock('^DJI', start, end)
    
    out =  pdr.data.get_data_yahoo(fout, start, end)
    out.columns.values[-1] = 'AdjClose'
    out.columns = out.columns + '_Out'
    out['Return_Out'] = out['AdjClose_Out'].pct_change()
    
    return [out, sp500, nasdaq, djia, frankfurt, london, paris, hkong, nikkei, australia]

d1 = datetime.datetime(2000,4,15)
d2 = datetime.datetime(2017,5,1)

bigDF = getStockDataFromWeb('GOOG',d1, d2)

             Open_^GSPC   High_^GSPC    Low_^GSPC  Close_^GSPC  Volume_^GSPC  \
Date                                                                           
2000-04-17  1356.560059  1401.530029  1346.500000  1401.439941    1204700000   
2000-04-18  1401.439941  1441.609985  1397.810059  1441.609985    1109400000   
2000-04-19  1441.609985  1447.689941  1424.260010  1427.469971    1001400000   
2000-04-20  1427.469971  1435.489990  1422.079956  1434.540039     896200000   
2000-04-24  1434.540039  1434.540039  1407.130005  1429.859985     868700000   
2000-04-25  1429.859985  1477.670044  1429.859985  1477.439941    1071100000   
2000-04-26  1477.439941  1482.939941  1456.979980  1460.989990     999600000   
2000-04-27  1460.989990  1469.209961  1434.810059  1464.920044    1111000000   
2000-04-28  1464.920044  1473.619995  1448.150024  1452.430054     984600000   
2000-05-01  1452.430054  1481.510010  1452.430054  1468.250000     966300000   
2000-05-02  1468.250000  1468.250000  14

In [4]:
def addFeatures(dataframe, adjclose, returns, n):
    """
    operates on two columns of dataframe:
    - n >= 2
    - given Return_* computes the return of day i respect to day i-n. 
    - given AdjClose_* computes its moving average on n days

    """
    
    return_n = adjclose[9:] + "Time" + str(n)
    dataframe[return_n] = dataframe[adjclose].pct_change(n)
    
    roll_n = returns[7:] + "RolMean" + str(n)
    dataframe[roll_n] = pd.DataFrame(dataframe[returns]).rolling(window = n).mean()



In [5]:
def applyRollMeanDelayedReturns(datasets, delta):
    """
    applies rolling mean and delayed returns to each dataframe in the list
    """
    for dataset in datasets:
        columns = dataset.columns    
        adjclose = columns[-2]
        returns = columns[-1]
        for n in delta:
            addFeatures(dataset, adjclose, returns, n)
    
    return datasets    

delta = {2}

h = applyRollMeanDelayedReturns(bigDF, delta)

In [6]:
h[0]

,Open_Out,High_Out,Low_Out,Close_Out,Volume_Out,AdjClose_Out,Return_Out,OutTime2,OutRolMean2
Date,,,,,,,,,
2004-08-19,100.000168,104.060182,95.960165,100.340176,44871300,50.119968,NaN,NaN,NaN
2004-08-20,101.010175,109.080187,100.500174,108.310183,22942800,54.100990,0.079430,NaN,NaN
2004-08-23,110.750191,113.480193,109.050183,109.400185,18342800,54.645447,0.010064,0.090293,0.044747
2004-08-24,111.240189,111.600192,103.570177,104.870176,15319700,52.382705,-0.041408,-0.031761,-0.015672
2004-08-25,104.960181,108.000187,103.880180,106.000184,9232100,52.947145,0.010775,-0.031079,-0.015316
2004-08-26,104.950180,107.950188,104.660179,107.910182,7128600,53.901190,0.018019,0.028988,0.014397
2004-08-27,108.100185,108.620186,105.690180,106.150181,6241200,53.022069,-0.016310,0.001415,0.000854
2004-08-30,105.280178,105.490184,102.010172,102.010172,5221400,50.954132,-0.039001,-0.054675,-0.027656
2004-08-31,102.300173,103.710180,102.160177,102.370175,4941200,51.133953,0.003529,-0.035610,-0.017736


In [7]:
def mergeDataframes(datasets, index):
    """
    merges datasets in the list 
    """
    subset = []#tion
    subset = [dataset.iloc[:, index:] for dataset in datasets[1:]]
    
    first = subset[0].join(subset[1:], how = 'outer')
    finance = datasets[0].iloc[:, index:].join(first, how = 'left') 
#    finance = finance[finance.index > cut]
    return finance

index = 6
"""
The cut is not useful
"""
#cut = datetime.datetime(2005,11,4)

mergedDF = mergeDataframes(bigDF, index)

mergedDF

,Return_Out,OutTime2,OutRolMean2,Return_^GSPC,^GSPCTime2,^GSPCRolMean2,Return_^IXIC,^IXICTime2,^IXICRolMean2,Return_^DJI,...,^FCHIRolMean2,Return_^HSI,^HSITime2,^HSIRolMean2,Return_^N225,^N225Time2,^N225RolMean2,Return_^AXJO,^AXJOTime2,^AXJORolMean2
Date,,,,,,,,,,,,,,,,,,,,,
2004-08-19,NaN,NaN,NaN,-0.003598,0.008801,0.004423,-0.006269,0.013725,0.006926,-0.004198,...,0.001597,0.013749,0.011468,0.005749,0.011998,0.016554,0.008250,0.002934,0.000890,0.000448
2004-08-20,0.079430,NaN,NaN,0.006525,0.002904,0.001464,0.009962,0.003631,0.001847,0.006904,...,-0.000488,-0.001595,0.012132,0.006077,-0.001320,0.010662,0.005339,0.003270,0.006214,0.003102
2004-08-23,0.010064,0.090293,0.044747,-0.002431,0.004078,0.002047,0.000370,0.010336,0.005166,-0.003669,...,0.006399,0.004433,0.002831,0.001419,0.006596,0.005268,0.002638,0.007519,0.010814,0.005395
2004-08-24,-0.041408,-0.031761,-0.015672,0.000465,-0.001967,-0.000983,-0.000984,-0.000615,-0.000307,0.002539,...,0.007955,0.017272,0.021782,0.010853,0.002222,0.008834,0.004409,0.000653,0.008177,0.004086
2004-08-25,0.010775,-0.031079,-0.015316,0.008000,0.008470,0.004233,0.012973,0.011976,0.005994,0.008230,...,0.000776,0.011587,0.029059,0.014430,0.013171,0.015423,0.007697,-0.002524,-0.001873,-0.000936
2004-08-26,0.018019,0.028988,0.014397,0.000118,0.008119,0.004059,-0.004192,0.008727,0.004391,-0.000818,...,0.004932,-0.000675,0.010904,0.005456,-0.000062,0.013108,0.006555,0.002445,-0.000085,-0.000039
2004-08-27,-0.016310,0.001415,0.000854,0.002425,0.002543,0.001271,0.004949,0.000736,0.000379,0.002123,...,0.007481,0.002662,0.001985,0.000993,0.007212,0.007149,0.003575,0.009331,0.011798,0.005888
2004-08-30,-0.039001,-0.054675,-0.027656,-0.007781,-0.005375,-0.002678,-0.013748,-0.008867,-0.004400,-0.007110,...,0.000955,0.004631,0.007305,0.003646,-0.002236,0.004960,0.002488,-0.001349,0.007969,0.003991
2004-08-31,0.003529,-0.035610,-0.017736,0.004631,-0.003187,-0.001575,0.000877,-0.012883,-0.006436,0.005078,...,-0.007550,-0.002135,0.002486,0.001248,-0.009186,-0.011401,-0.005711,-0.000394,-0.001742,-0.000871


In [8]:
def applyTimeLag(dataset, lags, delta):
    """
    apply time lag to return columns selected according  to delta.
    Days to lag are contained in the lags list passed as argument.
    Returns a NaN free dataset obtained cutting the lagged dataset
    at head and tail
    """
    
    dataset.Return_Out = dataset.Return_Out.shift(-1)
    
    maxLag = max(lags)
 
    allColumns = dataset.columns[:]
  
    for column in allColumns:
        data_col = pd.Series(dataset[column])
        data_col = data_col.interpolate(method='linear', axis=0, limit=None, inplace=False, limit_direction='forward', downcast=None)
        dataset[column] = data_col

    return dataset.iloc[maxLag:-1,:]

lags = {0, 1, 2}

hhh = applyTimeLag(mergedDF, lags, delta)

hhh

,Return_Out,OutTime2,OutRolMean2,Return_^GSPC,^GSPCTime2,^GSPCRolMean2,Return_^IXIC,^IXICTime2,^IXICRolMean2,Return_^DJI,...,^FCHIRolMean2,Return_^HSI,^HSITime2,^HSIRolMean2,Return_^N225,^N225Time2,^N225RolMean2,Return_^AXJO,^AXJOTime2,^AXJORolMean2
Date,,,,,,,,,,,,,,,,,,,,,
2004-08-23,-0.041408,0.090293,0.044747,-0.002431,0.004078,0.002047,0.000370,0.010336,0.005166,-0.003669,...,0.006399,0.004433,0.002831,0.001419,0.006596,0.005268,0.002638,0.007519,0.010814,0.005395
2004-08-24,0.010775,-0.031761,-0.015672,0.000465,-0.001967,-0.000983,-0.000984,-0.000615,-0.000307,0.002539,...,0.007955,0.017272,0.021782,0.010853,0.002222,0.008834,0.004409,0.000653,0.008177,0.004086
2004-08-25,0.018019,-0.031079,-0.015316,0.008000,0.008470,0.004233,0.012973,0.011976,0.005994,0.008230,...,0.000776,0.011587,0.029059,0.014430,0.013171,0.015423,0.007697,-0.002524,-0.001873,-0.000936
2004-08-26,-0.016310,0.028988,0.014397,0.000118,0.008119,0.004059,-0.004192,0.008727,0.004391,-0.000818,...,0.004932,-0.000675,0.010904,0.005456,-0.000062,0.013108,0.006555,0.002445,-0.000085,-0.000039
2004-08-27,-0.039001,0.001415,0.000854,0.002425,0.002543,0.001271,0.004949,0.000736,0.000379,0.002123,...,0.007481,0.002662,0.001985,0.000993,0.007212,0.007149,0.003575,0.009331,0.011798,0.005888
2004-08-30,0.003529,-0.054675,-0.027656,-0.007781,-0.005375,-0.002678,-0.013748,-0.008867,-0.004400,-0.007110,...,0.000955,0.004631,0.007305,0.003646,-0.002236,0.004960,0.002488,-0.001349,0.007969,0.003991
2004-08-31,-0.020709,-0.035610,-0.017736,0.004631,-0.003187,-0.001575,0.000877,-0.012883,-0.006436,0.005078,...,-0.007550,-0.002135,0.002486,0.001248,-0.009186,-0.011401,-0.005711,-0.000394,-0.001742,-0.000871
2004-09-01,0.012569,-0.017253,-0.008590,0.001512,0.006150,0.003072,0.006697,0.007580,0.003787,-0.000537,...,-0.003197,0.013509,0.011344,0.005687,0.004111,-0.005112,-0.002537,0.006446,0.006049,0.003026
2004-09-02,-0.014777,-0.008401,-0.004070,0.011213,0.012742,0.006362,0.012440,0.019221,0.009569,0.011980,...,0.005424,-0.001904,0.011579,0.005802,0.002283,0.006403,0.003197,0.004139,0.010612,0.005292


In [9]:
def prepareDataForCrossValidation(dataset):
    """
    generate categorical output column, set features and return whole data set
    """
    le = preprocessing.LabelEncoder()
    
    dataset['UpDown'] = dataset['Return_Out']
    dataset.UpDown[dataset.UpDown >= 0] = 'Up'
    dataset.UpDown[dataset.UpDown < 0] = 'Down'
    dataset.UpDown = le.fit(dataset.UpDown).transform(dataset.UpDown)
    """
    Here set feature from column 3 to the end, get rid of all the colomn related to S&P500
    """
    features = dataset.columns[3:-1]
    X = dataset[features]    
    y = dataset.UpDown
    return [X,y]

[X_whole, y_whole] = prepareDataForCrossValidation(hhh)
X_whole

C:\Users\ljia\Anaconda3\envs\py27\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\ljia\Anaconda3\envs\py27\lib\site-packages\pandas\core\generic.py:4702: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\Users\ljia\Anaconda3\envs\py27\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ljia\Anaco

,Return_^GSPC,^GSPCTime2,^GSPCRolMean2,Return_^IXIC,^IXICTime2,^IXICRolMean2,Return_^DJI,^DJITime2,^DJIRolMean2,Return_^GDAXI,...,^FCHIRolMean2,Return_^HSI,^HSITime2,^HSIRolMean2,Return_^N225,^N225Time2,^N225RolMean2,Return_^AXJO,^AXJOTime2,^AXJORolMean2
Date,,,,,,,,,,,,,,,,,,,,,
2004-08-23,-0.002431,0.004078,0.002047,0.000370,0.010336,0.005166,-0.003669,0.003210,0.001618,0.016034,...,0.006399,0.004433,0.002831,0.001419,0.006596,0.005268,0.002638,0.007519,0.010814,0.005395
2004-08-24,0.000465,-0.001967,-0.000983,-0.000984,-0.000615,-0.000307,0.002539,-0.001138,-0.000565,-0.000302,...,0.007955,0.017272,0.021782,0.010853,0.002222,0.008834,0.004409,0.000653,0.008177,0.004086
2004-08-25,0.008000,0.008470,0.004233,0.012973,0.011976,0.005994,0.008230,0.010790,0.005385,0.004741,...,0.000776,0.011587,0.029059,0.014430,0.013171,0.015423,0.007697,-0.002524,-0.001873,-0.000936
2004-08-26,0.000118,0.008119,0.004059,-0.004192,0.008727,0.004391,-0.000818,0.007405,0.003706,0.011455,...,0.004932,-0.000675,0.010904,0.005456,-0.000062,0.013108,0.006555,0.002445,-0.000085,-0.000039
2004-08-27,0.002425,0.002543,0.001271,0.004949,0.000736,0.000379,0.002123,0.001303,0.000653,0.004932,...,0.007481,0.002662,0.001985,0.000993,0.007212,0.007149,0.003575,0.009331,0.011798,0.005888
2004-08-30,-0.007781,-0.005375,-0.002678,-0.013748,-0.008867,-0.004400,-0.007110,-0.005002,-0.002494,-0.003202,...,0.000955,0.004631,0.007305,0.003646,-0.002236,0.004960,0.002488,-0.001349,0.007969,0.003991
2004-08-31,0.004631,-0.003187,-0.001575,0.000877,-0.012883,-0.006436,0.005078,-0.002069,-0.001016,-0.013973,...,-0.007550,-0.002135,0.002486,0.001248,-0.009186,-0.011401,-0.005711,-0.000394,-0.001742,-0.000871
2004-09-01,0.001512,0.006150,0.003072,0.006697,0.007580,0.003787,-0.000537,0.004538,0.002271,0.008562,...,-0.003197,0.013509,0.011344,0.005687,0.004111,-0.005112,-0.002537,0.006446,0.006049,0.003026
2004-09-02,0.011213,0.012742,0.006362,0.012440,0.019221,0.009569,0.011980,0.011437,0.005722,0.004147,...,0.005424,-0.001904,0.011579,0.005802,0.002283,0.006403,0.003197,0.004139,0.010612,0.005292


In [10]:
def performClassification(X_train, y_train, X_test, y_test, method):
    """
    performs classification on daily returns using several algorithms (method).
    method --> string algorithm
    parameters --> list of parameters passed to the classifier (if any)
    fout --> string with name of stock to be predicted
    savemodel --> boolean. If TRUE saves the model to pickle file
    """
   
    if method == 'RF':   
        return performRFClass(X_train, y_train, X_test, y_test)
        
    elif method == 'KNN':
        return performKNNClass(X_train, y_train, X_test, y_test)
    
    elif method == 'SVM':   
        return performSVMClass(X_train, y_train, X_test, y_test)
    
    elif method == 'ADA':
        return performAdaBoostClass(X_train, y_train, X_test, y_test)
    
    elif method == 'GTB': 
        return performGTBClass(X_train, y_train, X_test, y_test)

    elif method == 'QDA': 
        return performQDAClass(X_train, y_train, X_test, y_test)

In [11]:
def performRFClass(X_train, y_train, X_test, y_test):
    """
    Random Forest Binary Classification
    """
    clf = RandomForestClassifier(n_estimators=1000, n_jobs=-1)
    clf.fit(X_train, y_train)

    feature_importance = clf.feature_importances_
    print feature_importance
    accuracy = clf.score(X_test, y_test)
    
        
    y_predict = clf.predict(X_test)
    
    print("y_predict = ", y_predict)
        
    tn, fp, fn, tp = confusion_matrix(y_test, y_predict).ravel()
    print(tn, fp, fn, tp)
    
    target_names = ['Down', 'Up']
    print(classification_report(y_test, y_predict, target_names=target_names))
    
    return accuracy

#performRFClass (X_train, y_train, X_test, y_test)

In [12]:
def performKNNClass(X_train, y_train, X_test, y_test):
    """
    KNN binary Classification
    """
    clf = neighbors.KNeighborsClassifier()
    clf.fit(X_train, y_train)
    
    accuracy = clf.score(X_test, y_test)
            
    y_predict = clf.predict(X_test)    
    print 'y_predict = ', y_predict
        
    tn, fp, fn, tp = confusion_matrix(y_test, y_predict).ravel()
    print '(tn, fp, fn, tp) : (', tn, fp, fn,tp , ')'
    
    target_names = ['Down', 'Up']
    print(classification_report(y_test, y_predict, target_names=target_names))
    
    
    return accuracy
#performKNNClass (X_train, y_train, X_test, y_test)

In [13]:
def performSVMClass(X_train, y_train, X_test, y_test):
    """
    SVM binary Classification
    """
    clf = SVC()
    clf.fit(X_train, y_train)

    accuracy = clf.score(X_test, y_test)
    
    y_predict = clf.predict(X_test)
    print 'y_predict = ', y_predict
        
    tn, fp, fn, tp = confusion_matrix(y_test, y_predict).ravel()
    print '(tn, fp, fn, tp) : (', tn, fp, fn,tp , ')'
    
    target_names = ['Down', 'Up']
    print(classification_report(y_test, y_predict, target_names=target_names))
    
    return accuracy
#performSVMClass (X_train, y_train, X_test, y_test)

In [14]:
def performAdaBoostClass(X_train, y_train, X_test, y_test):
    """
    Ada Boosting binary Classification
    """
    clf = AdaBoostClassifier()
    clf.fit(X_train, y_train)
    
    accuracy = clf.score(X_test, y_test)

    y_predict = clf.predict(X_test)
    print 'y_predict = ', y_predict
        
    tn, fp, fn, tp = confusion_matrix(y_test, y_predict).ravel()
    print '(tn, fp, fn, tp) : (', tn, fp, fn,tp , ')'
    
    target_names = ['Down', 'Up']
    print(classification_report(y_test, y_predict, target_names=target_names))
    
    return accuracy
#performAdaBoostClass (X_train, y_train, X_test, y_test)

In [15]:
def performGTBClass(X_train, y_train, X_test, y_test):
    """
    Gradient Tree Boosting binary Classification
    """
    clf = GradientBoostingClassifier(n_estimators=100)
    clf.fit(X_train, y_train)
    
    y_predict = clf.predict(X_test)
    print 'y_predict = ', y_predict
        
    tn, fp, fn, tp = confusion_matrix(y_test, y_predict).ravel()
    print'(tn, fp, fn, tp) : (', tn, fp, fn,tp , ')'
    
    target_names = ['Down', 'Up']
    print(classification_report(y_test, y_predict, target_names=target_names))
    
    accuracy = clf.score(X_test, y_test)
    
    return accuracy
#performGTBClass(X_train, y_train, X_test, y_test)

In [16]:
def performQDAClass(X_train, y_train, X_test, y_test):
    """
    Quadratic Discriminant Analysis binary Classification
    """
   
    clf = QDA()
    clf.fit(X_train, y_train)
    
    accuracy = clf.score(X_test, y_test)
    y_predict = clf.predict(X_test)
    print 'y_predict = ', y_predict
        
    tn, fp, fn, tp = confusion_matrix(y_test, y_predict).ravel()
    print '(tn, fp, fn, tp) : (', tn, fp, fn,tp , ')'
    
    target_names = ['Down', 'Up']
    print(classification_report(y_test, y_predict, target_names=target_names))
    
    return accuracy

#performQDAClass(X_train, y_train, X_test, y_test)

In [17]:
def timeSeriesCV(dataset_X,dataset_y, n_splits, algorithm):
    print dataset_X.shape
    
    tscv = TimeSeriesSplit(n_splits=n_splits)
    counter = 0
    accuracies = np.zeros(n_splits)
    print accuracies
    for train_index, test_index in tscv.split(dataset_X):
        counter += 1
        print 'cyle: ', counter
        print 'TRAIN size: ', train_index[-1], 'TEST size: ', test_index[-1] - test_index[0] 
    
        X_train, X_test = dataset_X[:train_index[-1]], dataset_X[test_index[0]:test_index[-1]]
        y_train, y_test = dataset_y[:train_index[-1]], dataset_y[test_index[0]:test_index[-1]]
        
        print 'X_train shape: ', X_train.shape
        print 'X_test shape: ', X_test.shape
        
        index = counter - 1
        accuracies[index] = performClassification(X_train, y_train, X_test, y_test, algorithm)
        print 'Accuracy: ', accuracies[index], '\n'
        
    print 'Average accuracy: ', np.mean(accuracies)
    return np.mean(accuracies)

    
timeSeriesCV(X_whole, y_whole, 10, 'RF')

(3194, 27)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
cyle:  1
TRAIN size:  293 TEST size:  289
X_train shape:  (293, 27)
X_test shape:  (289, 27)
[ 0.03240946  0.03811993  0.04078408  0.04205913  0.04403848  0.0432943
  0.03679887  0.0351514   0.03615942  0.05244047  0.03630633  0.03428176
  0.03803799  0.0290666   0.03221392  0.03762695  0.02984282  0.03054276
  0.03854371  0.03169166  0.0300229   0.04201433  0.03750624  0.03579237
  0.03940835  0.03889293  0.03695282]
('y_predict = ', array([1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 

0.5017301038062284

In [20]:
timeSeriesCV(X_whole, y_whole, 10, 'SVM')

(3194, 27)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
cyle:  1
TRAIN size:  293 TEST size:  289
X_train shape:  (293, 27)
X_test shape:  (289, 27)
y_predict =  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
(tn, fp, fn, tp) : ( 0 143 0 146 )
             precision    recall  f1-score   support

       Down       0.00      0.00      0.00       143
         Up       0.51      1.00      0.67       146

avg / total       0.26      0.51      0.34       289

Accu

0.51072664359861586

In [19]:
timeSeriesCV(X_whole, y_whole, 10, 'ADA')

(3194, 27)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
cyle:  1
TRAIN size:  293 TEST size:  289
X_train shape:  (293, 27)
X_test shape:  (289, 27)
y_predict =  [1 1 0 1 0 0 1 0 1 0 0 1 1 1 1 0 0 1 1 1 0 0 1 1 1 1 0 1 1 0 0 0 1 1 1 1 1
 0 0 0 1 0 0 1 1 0 1 1 0 1 1 0 0 1 0 1 1 0 1 0 0 1 1 0 1 1 1 1 0 1 1 1 0 1
 1 1 0 1 0 1 1 1 1 0 0 1 1 1 1 0 1 1 1 0 1 0 0 0 0 1 1 1 0 0 1 1 1 1 0 0 1
 1 1 0 0 0 0 1 0 0 1 1 0 1 0 0 1 1 1 1 0 0 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0
 1 0 0 1 0 0 0 1 1 0 0 0 0 1 0 0 1 0 1 1 0 1 1 0 1 0 1 1 0 0 1 0 1 1 1 1 0
 0 1 1 0 1 0 1 0 0 0 0 0 1 0 1 1 0 1 1 1 0 1 0 0 1 1 0 1 1 0 1 1 0 1 0 0 0
 1 1 1 1 0 1 1 0 0 1 1 0 1 0 0 1 0 1 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 1 1 0 1
 1 0 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 0 0 0 1 0 0 1 1 1 1]
(tn, fp, fn, tp) : ( 57 86 63 83 )
             precision    recall  f1-score   support

       Down       0.47      0.40      0.43       143
         Up       0.49      0.57      0.53       146

avg / total       0.48      0.48      0.48       289

Accu

0.4865051903114187

In [21]:
timeSeriesCV(X_whole, y_whole, 10, 'GTB')

(3194, 27)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
cyle:  1
TRAIN size:  293 TEST size:  289
X_train shape:  (293, 27)
X_test shape:  (289, 27)
y_predict =  [1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 1 1 1 0 0 1 0 0 0 1 0 1 1 1
 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 0 1 0 0 1 0 1 1 1 1 0 1 0 1
 1 1 0 1 0 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 1 0 0 1 0 1 1 1 0 1 1 1 1 0 1 1 1
 1 1 0 1 1 0 0 0 1 1 1 1 0 1 1 1 1 1 1 0 0 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0
 1 1 1 1 0 1 0 0 1 0 1 0 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 0 0 1 0 1 1 1 1 0
 1 1 1 0 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1 0 0 1 1 0 0 0 0 1 1 0 1 1 0 1 1 0 1
 1 1 0 1 0 1 1 0 0 1 1 1 1 1 0 1 0 1 0 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1
 0 1 0 1 0 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 0 0 1 1 0 0 1 1 1 1]
(tn, fp, fn, tp) : ( 48 95 44 102 )
             precision    recall  f1-score   support

       Down       0.52      0.34      0.41       143
         Up       0.52      0.70      0.59       146

avg / total       0.52      0.52      0.50       289

Acc

0.50276816608996533

In [22]:
timeSeriesCV(X_whole, y_whole, 10, 'QDA')

(3194, 27)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
cyle:  1
TRAIN size:  293 TEST size:  289
X_train shape:  (293, 27)
X_test shape:  (289, 27)
y_predict =  [1 1 1 1 0 1 1 1 1 0 1 1 1 1 0 1 0 0 0 1 1 1 0 1 0 1 0 0 0 1 1 1 0 1 1 1 1
 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 0 1 0 1 1 1
 0 1 1 0 0 1 1 1 1 1 0 1 1 1 1 0 1 1 1 0 1 1 1 0 1 1 0 1 1 0 0 1 0 1 1 0 1
 1 0 0 1 1 0 1 1 1 0 1 1 1 0 0 0 1 0 1 0 1 0 1 1 1 1 0 1 0 0 1 1 1 1 1 0 0
 1 0 0 0 0 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 0 1 0 1 1
 1 0 1 1 0 1 1 1 1 1 0 1 1 0 1 1 1 1 0 0 1 1 0 0 0 1 0 1 1 0 0 1 1 0 0 1 0
 0 0 0 1 1 1 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0 1 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1
 1 0 0 1 0 0 1 0 0 1 1 0 0 1 1 1 1 1 0 1 1 1 1 0 0 0 0 1 1 0]
(tn, fp, fn, tp) : ( 46 97 71 75 )
             precision    recall  f1-score   support

       Down       0.39      0.32      0.35       143
         Up       0.44      0.51      0.47       146

avg / total       0.41      0.42      0.41       289

Accu

C:\Users\ljia\Anaconda3\envs\py27\lib\site-packages\sklearn\discriminant_analysis.py:695: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


1163 TEST size:  289
X_train shape:  (1163, 27)
X_test shape:  (289, 27)
y_predict =  [1 1 0 0 1 0 0 1 1 1 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 1 1 1 1 0 0 0 1 1 1 1 0
 1 1 0 0 1 1 1 1 1 1 1 1 1 0 1 0 0 1 0 1 1 1 0 0 0 1 1 1 1 1 0 1 1 1 1 1 1
 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1
 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 1 0 1 1 1 0 1 0 0]
(tn, fp, fn, tp) : ( 26 109 32 122 )
             precision    recall  f1-score   support

       Down       0.45      0.19      0.27       135
         Up       0.53      0.79      0.63       154

avg / total       0.49      0.51      0.46       289

Accuracy:  0.512110726644 

cyle:  5
TRAIN size:  1453 TEST size:  289
X_tr

0.49757785467128024

In [23]:
timeSeriesCV(X_whole, y_whole, 10, 'KNN')

(3194, 27)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
cyle:  1
TRAIN size:  293 TEST size:  289
X_train shape:  (293, 27)
X_test shape:  (289, 27)
y_predict =  [1 1 0 1 1 0 1 1 0 1 1 1 1 0 1 0 1 0 1 1 1 1 1 0 1 0 1 1 1 0 1 1 1 0 1 1 1
 1 0 1 0 1 0 0 1 1 1 0 0 1 0 1 1 0 0 1 1 1 1 1 0 1 1 0 0 0 0 0 1 1 0 1 0 1
 1 1 1 1 0 0 0 1 1 1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 0 0 1 1 1 1 1 0 1 1 0 1 1
 1 1 0 0 1 0 1 0 0 0 1 0 1 1 1 1 0 1 1 0 1 1 0 1 1 1 1 0 0 0 0 0 0 0 1 0 1
 0 0 0 0 0 1 1 0 0 0 1 1 0 1 1 1 1 0 1 0 0 0 0 1 1 1 0 0 1 1 1 0 0 1 0 1 0
 1 1 0 0 0 1 1 0 0 1 0 1 1 0 1 0 0 1 1 0 0 1 1 1 0 0 0 1 0 0 0 1 0 1 1 0 0
 1 1 1 1 0 1 1 1 0 1 0 0 1 1 0 1 0 1 0 1 0 0 1 1 0 1 1 0 1 0 1 0 1 1 1 1 1
 1 1 0 1 0 1 1 0 1 1 1 0 1 0 0 0 1 1 0 0 1 1 1 1 1 1 0 1 1 1]
(tn, fp, fn, tp) : ( 64 79 61 85 )
             precision    recall  f1-score   support

       Down       0.51      0.45      0.48       143
         Up       0.52      0.58      0.55       146

avg / total       0.52      0.52      0.51       289

Accu

0.49446366782006923